# LangChain: Models, Prompts and Output Parsers

In [100]:
from L1 import *

### Openai

For simple calls

In [57]:
simple_prompt = "What is 1+1?"

complete(simple_prompt)

'As an AI language model, I can tell you that the answer to 1+1 is 2.'

For generic prompts

In [58]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

complete(prompt)

'I am quite upset that my blender lid came off and caused my smoothie to splatter all over my kitchen walls. Additionally, the warranty does not cover the cost of cleaning up the mess. Would you be able to assist me at this time, my friend?'

### LangChain

For simple calls

In [59]:
chat = ChatOpenAI(temperature = 0.0, model = llm_model)

print(chat.invoke("Hello"))
print(chat.invoke("World"))

content='Hello! How can I assist you today?' response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo-0301', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d3ca09cd-564c-4f05-9637-654c8455d9e1-0'
content='The world is the planet Earth and all the people, animals, plants, and other living things that inhabit it. It is a complex and diverse place, with a wide range of cultures, languages, religions, and ecosystems. The world is constantly changing, with new discoveries, technologies, and challenges emerging every day. Despite its many differences, the world is also interconnected, with events and actions in one part of the globe affecting people and places far away. As such, it is important for individuals and societies to work together to address global issues and create a more sustainable and equitable world for all.' response_metadata={'token_usage': {'completion_tokens': 119, 'prom

In [60]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = new_template(template_string)
print(prompt_template[0].prompt)

customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = new_prompt(prompt_template, 
    style=customer_style,
    text=customer_email,
)
print(customer_messages[0].content)

customer_response = chat.invoke(customer_messages)
print(customer_response.content)

input_variables=['style', 'text'] template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'
Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone
. text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! To add insult to injury, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?


In [61]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate
"""

service_messages = new_prompt(prompt_template,
    style=service_style_pirate,
    text=service_reply,
)
print(service_messages[0].content)

service_response = chat(service_messages)
print(service_response.content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```

Ahoy there, matey! I must kindly inform ye that the warranty be not coverin' the expenses o' cleaning yer galley, as 'tis yer own fault fer misusin' yer blender by forgettin' to put the lid on afore startin' it. Aye, tough luck! Farewell and may the winds be in yer favor!


For returning a structured JSON output

In [62]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings: \
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, \
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

prompt_template = new_template(review_template)
print(prompt_template.messages[0])

messages = new_prompt(prompt_template, 
    text=customer_review,
)

response = chat.invoke(messages)
print(response.content)

prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n')
{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


For returning the JSON output in a safer manner

In [98]:
gift_description = "Was the item purchased \
as a gift for someone else? \
Answer True if yes, \
False if not or unknown."

delivery_days_description = "How many days \
did it take for the product \
to arrive? If this \
information is not found, \
output -1."

price_value_description="Extract any \
sentences about the value or \
price, and output them as a \
comma separated Python list."

schema_args = [
    dict(
        name="gift", 
        description=gift_description,
        type="bool",
    ),
    dict(
        name="delivery_days", 
        description=delivery_days_description, 
        type="int",
    ),
    dict(
        name="price_value",
        description=price_value_description,
        type="string",
    )
]

output_parser = new_output_parser(*schema_args)
parser_prompt = get_parser_prompt(output_parser)
print(parser_prompt)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": bool  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": int  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


In [99]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, \
and output them as a comma separated Python list.

text: {text}

{parser_prompt}
"""

template = new_template(review_template_2)

messages = new_prompt(template,
    text=customer_review, 
    parser_prompt=parser_prompt,
)
print(messages[0].content)

response = chat.invoke(messages)
print(response.content)

output_dict = output_parser.parse(response.content)
print(output_dict)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```